# FourSquare API Tutorial
### Jordan Ireland

In [1]:
## install a few things we may need. This may take a few minutes
!pip install ipywidgets python-decouple
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter nbextension enable --py widgetsnbextension

  Created wheel for python-decouple: filename=python_decouple-3.3-cp37-none-any.whl size=9034 sha256=7e2d09389a06971167a9364e6195600c841b009244ae5fd273f77a7376dfa8b4
  Stored in directory: C:\Users\pandu\AppData\Local\pip\Cache\wheels\f4\9b\79\5eec9ea205382552b014e749cb8a2b4ce0d6836a6f4db5ac0c
Successfully built python-decouple


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


An error occured.
RuntimeError: JupyterLab failed to build
See the log file for details:  C:\Users\pandu\AppData\Local\Temp\jupyterlab-debug-sakcc7rs.log


Building jupyterlab assets
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


#### What are we doing up there?
**ipywidgets** makes your notebooks interactable by allowing you to add forms, sliders, buttons, etc. They're a little in depth, but I will cover the basics here! \
The next few lines allows them to work inside Jupyter Labs. I don't know how they work, but they work!

**decouple** is a way to keep secret codes and keys from being exposed. Github will ignore them so they won't be uploaded and adds some security to python. \
In order to use it. You have to make a `.env` file inside of your base directory for that project. (.env is the full file name. Make sure there are no spaces before it).
I will go over this later on.

### Make a Foursquare Developer Account
[Sign up for Foursquare](https://foursquare.com/developers/signup)

Click `Create New App`, Name it whatever you want! I'm naming my 'Foursquare API Tutorial'. The website can be your portfolio link!

![appsetup.png](appsetup.png)

#### Once you're done, don't share your Client ID or Client Secret with anyone.

## Creating some basic widgets to get the values we want

In [1]:
## import things we need
import ipywidgets as widgets
import requests
import pandas as pd
import numpy as np
from decouple import config

## Create class so ipywidgets can update values after running the cell
class Updated:

    def __init__(self, city, radius, r_type):
        self.city_value = city
        self.radius_value = radius
        self.type_value = r_type

    def update(self, city, radius, r_type) -> None:
        self.city_value = city.value
        self.radius_value = radius.value
        self.type_value = r_type.value

## Create widgets so we can interact with the API
city_input = widgets.Dropdown(
    options=['Phoenix', 'Chicago', 'San Diego', 'New York City']
)
type_input = widgets.Dropdown(
    options= [('BBQ Joint', '4bf58dd8d48988d1df931735'), ('American', '4bf58dd8d48988d14e941735'), ('Asian', '4bf58dd8d48988d142941735')]
)
radius_input = widgets.IntSlider(
    value=50000,
    min=1000,
    max=100000,
    step=250,
    description='Range:',
    continuous_update=False,
    orientation='horizontal'
)
button = widgets.Button(description="Submit")
output = widgets.Output()

## Make function for buttom being clicked
def on_button_clicked(b):
    with output:
        query.update(city_input, radius_input, type_input)

button.on_click(on_button_clicked)
display(city_input, radius_input, type_input, button, output)

## Make a new class with default values
query = Updated(city_input.value, radius_input.value, type_input.value)

## URL to foursquare's API
URL = 'https://api.foursquare.com/v2/venues/search'


Dropdown(options=('Phoenix', 'Chicago', 'San Diego', 'New York City'), value='Phoenix')

IntSlider(value=50000, continuous_update=False, description='Range:', max=100000, min=1000, step=250)

Dropdown(options=(('BBQ Joint', '4bf58dd8d48988d1df931735'), ('American', '4bf58dd8d48988d14e941735'), ('Asian…

Button(description='Submit', style=ButtonStyle())

Output()

## Calling the API

An API GET request is broken into a few things.
* The base URL, in this case it's `https://api.foursquare.com/v2/venues/search`
* The query string. A query string is a list of variables and values that are set by the API you're using. [View Foursquares API Variables](https://developer.foursquare.com/docs/api/venues/search) If you see, there are a few required ones. `ll` OR `near`. The others are optional. How do we use them?
 * After the base url, you need to add a `?`. That tells the browser where the actual request starts with custom variables.
   * `https://api.foursquare.com/v2/venues/search?`
 * Enter your first Variable
   * `https://api.foursquare.com/v2/venues/search?near=Chicago`
 * Separate your variables with a `&`.
   * `https://api.foursquare.com/v2/venues/search?near=Chicago&radius=25000`
   
That's a lot of work for 1 request. Luckily requests has a parameter! \
First you have to make a dictionary with all the values you want. `'near':'Chicago'` is the same as above! 

Lets put it into practice

In [2]:
## Make a new dictionary for Python's request GET
PARAMS = {'near':query.city_value, 'radius':query.radius_value, 'categoryId':query.type_value, 'client_id':config('CLIENT_ID'), 'client_secret':config('CLIENT_SEC'), 'v':'20180412'} 

## sending get request and saving the response as response object 
r = requests.get(url = URL, params = PARAMS) 

## extracting data in json format
response = r.json()

## This is what we get from the API
response

{'meta': {'code': 200, 'requestId': '5dd814d3ad1ab4001b672682'},
 'response': {'venues': [{'id': '5b3717439deb7d0039e963a1',
    'name': 'Rudy\'s "Country Store" and Bar-B-Q',
    'contact': {},
    'location': {'address': '1733 N Higley Rd',
     'lat': 33.3814058,
     'lng': -111.7210321,
     'labeledLatLngs': [{'label': 'display',
       'lat': 33.3814058,
       'lng': -111.7210321}],
     'postalCode': '85234',
     'cc': 'US',
     'city': 'Gilbert',
     'state': 'AZ',
     'country': 'United States',
     'formattedAddress': ['1733 N Higley Rd',
      'Gilbert, AZ 85234',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1df931735',
      'name': 'BBQ Joint',
      'pluralName': 'BBQ Joints',
      'shortName': 'BBQ',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
       'suffix': '.png'},
      'primary': True}],
    'verified': False,
    'stats': {'tipCount': 0,
     'usersCount': 0,
     'checkinsCount': 0,
     'visitsCou

## Breaking down the response

In [20]:
print(f"Meta: {response['meta']}")
print(f"Response Code: {response['meta']['code']}")
print(f"Request ID: {response['meta']['requestId']}")

Meta: {'code': 200, 'requestId': '5dd80f0d71782e001be79141'}
Response Code: 200
Request ID: 5dd80f0d71782e001be79141


* Meta is the metadata from the API response. It tells you about the call
* The response code is what happened during the call. Code 200 usually means that the resquest for information was successful and there weren't any errors. This **doesn't** mean you got the data you wanted. Just that it didn't fail
* Most APIs give you a request ID that you can look up (through another API call to a separate endpoint), to look up more information.

If you want to see Foursquares Codes, go [here](https://developer.foursquare.com/docs/api/troubleshooting/errors)

To view a pretty version or the JSON file at anytime, view the [example tree](https://codebeautify.org/jsonviewer/cb8187b7)

In [21]:
## This is what we really care about
data = response['response']
data

{'venues': [{'id': '51bd1640498e9a4f3c67a5c8',
   'name': 'Gaslamp BBQ',
   'contact': {},
   'location': {'address': '524 Island Ave',
    'lat': 32.710492957951836,
    'lng': -117.15982530474932,
    'labeledLatLngs': [{'label': 'display',
      'lat': 32.710492957951836,
      'lng': -117.15982530474932}],
    'postalCode': '92101',
    'cc': 'US',
    'neighborhood': 'Gaslamp',
    'city': 'San Diego',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['524 Island Ave',
     'San Diego, CA 92101',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d116941735',
     'name': 'Bar',
     'pluralName': 'Bars',
     'shortName': 'Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
      'suffix': '.png'},
     'primary': True}],
   'verified': True,
   'stats': {'tipCount': 0,
    'usersCount': 0,
    'checkinsCount': 0,
    'visitsCount': 0},
   'beenHere': {'count': 0,
    'lastCheckinExpiredAt': 0,
    'marked'

#### File Structure of *this* JSON response

Venues \
├── ID \
├── Name \
├── Contact \
└── Location \
. . . . .├── Latitude \
. . . . .├── Longitude \
. . . . .├── Postal Code \
. . . . .├── City \
. . . . .└── State

#### Cool. So how do we access this data?
All you have to do is treat it like a panda dataframe! \
`data['venues']` gives you a list of all the Restaurants that it returned \
Lets use that to make a `for` loop to iterate through all the restaurants and use 'name' value to get the name!

In [22]:
for restaurant in data['venues']:
    print(restaurant['name'])

Gaslamp BBQ
Phil's BBQ
Phil's BBQ
Phil's BBQ
9175 Judicial BBQ
Grand Ole BBQ Flinn Springs
Del's Hideout
The Pioneer
Wood Ranch BBQ & Grill
Phil's BBQ
Iron Pig Alehouse
Sam Woo BBQ
Phil's BBQ
The Spot
Mike's BBQ
West Coast BBQ and Brew
Kogi BBQ
Manna BBQ
Gyu-Kaku Japanese BBQ
Carnitas' Snack Shack - Embarcadero
BT's Southern BBQ
Kansas City Barbeque
Cheers Bar & Grill
Ranchwood BBQ
Tacos Kike
Bubba's Smokehouse
Coaster Saloon
Calypso Bay Smokehouse
Bull's Smokin BBQ
Lil' Piggy's Bar-B-Q


### Let's put it into a Pandas dataframe

In [23]:
## Create a dictionary with the a list as the value
restaurant_dict = {'id':[],'name':[],'category':[],'category_id':[]}

## Loop through each retaurant and append the respective fields
for restaurant in data['venues']:
    restaurant_dict['id'].append(restaurant['id'])
    restaurant_dict['name'].append(restaurant['name'])
    restaurant_dict['category'].append(restaurant['categories'][0]['name'])
    restaurant_dict['category_id'].append(restaurant['categories'][0]['id'])
    
## Make a pandas dataframe with the dictionary    
df = pd.DataFrame.from_dict(restaurant_dict)

restaurant_dict

{'id': ['51bd1640498e9a4f3c67a5c8',
  '5203ccf0498e4458881c39d3',
  '460d2cfcf964a52002451fe3',
  '57195c6e498e3e95890ce3f3',
  '51412755e4b0096f798128b6',
  '5ba707e03e6741002c41e911',
  '5ca29c525a2c910039f05ac7',
  '5b305ff1efa82a002cc219af',
  '554ac835498efb9e60cfa4c9',
  '4c6d440565eda093b30c4ed0',
  '5452dc71498ea5abf9a31300',
  '4ab5a433f964a520ba7520e3',
  '5074a839e4b07ac4790dadff',
  '4a836510f964a52004fb1fe3',
  '4e078f4eb0fbba5eee1d0560',
  '4c0c4805a1b32d7f3e659cf0',
  '4fdc1081e4b0b4dd28be2c98',
  '5474e871498efe80cf40b0b6',
  '52b17b66498eeab2c01f23f1',
  '572a5d9ecd105e31e87c7034',
  '566b3764498e25d72e3a6bb3',
  '40e0b100f964a52051021fe3',
  '52071b7b11d2dfe8a4be8a1b',
  '5cf96e7d1acf11002c15294f',
  '56ef801d498e353289c0b8a7',
  '4abd4e23f964a520a08920e3',
  '40e0b100f964a520f3011fe3',
  '4bfc2322e05e0f470c40cfa8',
  '4b4d21e4f964a5201ccc26e3',
  '4a9054fcf964a520551720e3'],
 'name': ['Gaslamp BBQ',
  "Phil's BBQ",
  "Phil's BBQ",
  "Phil's BBQ",
  '9175 Judicial BBQ

In [24]:
df.head()

,id,name,category,category_id
0,51bd1640498e9a4f3c67a5c8,Gaslamp BBQ,Bar,4bf58dd8d48988d116941735
1,5203ccf0498e4458881c39d3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735
2,460d2cfcf964a52002451fe3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735
3,57195c6e498e3e95890ce3f3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735
4,51412755e4b0096f798128b6,9175 Judicial BBQ,BBQ Joint,4bf58dd8d48988d1df931735


Awesome! Now let's try and add the address to the dataframe.

We'll use the same way. Use the `for` loop to go through them all.

In [25]:
addresses = []

for restaurant in data['venues']:
    addresses.append(restaurant['location']['address'])

KeyError: 'address'

Why didn't this work? \

Sometimes when a result doesn't have the value specified, like a food truck, *some* APIs will just not send you anything instead of sending you a blank key/value pair.\
So what's the point, if you can't use a for loop? \

Lucky for us, we can create something to give us a NaN value!

In [26]:
addresses = []

for restaurant in data['venues']:
    try:
        ## Set it to the address
        addresses.append(restaurant["location"]['address'])
    except KeyError:
        ## Set it to NaN
        addresses.append(np.NaN)
        
addresses

['524 Island Ave',
 'Terminal 2',
 '3740 Sports Arena Blvd',
 '17051 W Bernardo Dr',
 nan,
 '15505 Olde Highway 80',
 '5351 Adobe Falls Rd',
 '8622 Lake Murray Blvd',
 '7510 Hazard Center Dr. Unit 215',
 '579 Grand Ave',
 '1520 Garnet Ave',
 '7330 Clairemont Mesa Blvd',
 '9816 Mission Gorge Rd',
 '1005 Prospect St',
 '1356 W Valley Pkwy',
 '6126 Lake Murray Blvd',
 '5447 Kearny Villa Rd',
 '8188 Mira Mesa Blvd',
 '9844 Hibert St Ste G-1',
 '1004 N Harbor Dr',
 '7845 Highland Village Pl # C101',
 '600 W Harbor Dr',
 '9995 Carmel Mountain Rd Ste B1',
 '2533 Folex Way',
 'Panamericano',
 '888 Prospect St',
 '744 Ventura Pl',
 '500 Sea World Dr',
 '1127 W Morena Blvd',
 '1201 1st St']

Time to add to the dataframe!

In [27]:
df['address'] = addresses

print(df.isna().sum())
df.head()

id             0
name           0
category       0
category_id    0
address        1
dtype: int64


,id,name,category,category_id,address
0,51bd1640498e9a4f3c67a5c8,Gaslamp BBQ,Bar,4bf58dd8d48988d116941735,524 Island Ave
1,5203ccf0498e4458881c39d3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735,Terminal 2
2,460d2cfcf964a52002451fe3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735,3740 Sports Arena Blvd
3,57195c6e498e3e95890ce3f3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735,17051 W Bernardo Dr
4,51412755e4b0096f798128b6,9175 Judicial BBQ,BBQ Joint,4bf58dd8d48988d1df931735,NaN


That's a lot of work, so let's make a function to handle the error stuff for us.

In [28]:
def check_errors(json_data, key_name):
    try:
        ## Set it to the the key value
        return json_data[key_name]
    except KeyError:
        ## Set it to NaN
        return np.NaN

Now to add the zip code, state, and city

In [29]:
cities = []
zips = []
states = []

for restaurant in data['venues']:
    cities.append(check_errors(restaurant['location'],'city'))
    zips.append(check_errors(restaurant['location'],'postalCode'))
    states.append(check_errors(restaurant['location'],'state'))

In [30]:
df['citiy'], df['state'], df['zip_code'] = [cities, states, zips]

df.head()

,id,name,category,category_id,address,citiy,state,zip_code
0,51bd1640498e9a4f3c67a5c8,Gaslamp BBQ,Bar,4bf58dd8d48988d116941735,524 Island Ave,San Diego,CA,92101
1,5203ccf0498e4458881c39d3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735,Terminal 2,San Diego,CA,92110
2,460d2cfcf964a52002451fe3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735,3740 Sports Arena Blvd,San Diego,CA,92110
3,57195c6e498e3e95890ce3f3,Phil's BBQ,BBQ Joint,4bf58dd8d48988d1df931735,17051 W Bernardo Dr,San Diego,CA,92127
4,51412755e4b0096f798128b6,9175 Judicial BBQ,BBQ Joint,4bf58dd8d48988d1df931735,NaN,San Diego,CA,NaN


### Your turn!

Add the column 'Verified' to dataframe using our function!

In [31]:
## Your code here
data

{'venues': [{'id': '51bd1640498e9a4f3c67a5c8',
   'name': 'Gaslamp BBQ',
   'contact': {},
   'location': {'address': '524 Island Ave',
    'lat': 32.710492957951836,
    'lng': -117.15982530474932,
    'labeledLatLngs': [{'label': 'display',
      'lat': 32.710492957951836,
      'lng': -117.15982530474932}],
    'postalCode': '92101',
    'cc': 'US',
    'neighborhood': 'Gaslamp',
    'city': 'San Diego',
    'state': 'CA',
    'country': 'United States',
    'formattedAddress': ['524 Island Ave',
     'San Diego, CA 92101',
     'United States']},
   'categories': [{'id': '4bf58dd8d48988d116941735',
     'name': 'Bar',
     'pluralName': 'Bars',
     'shortName': 'Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/pub_',
      'suffix': '.png'},
     'primary': True}],
   'verified': True,
   'stats': {'tipCount': 0,
    'usersCount': 0,
    'checkinsCount': 0,
    'visitsCount': 0},
   'beenHere': {'count': 0,
    'lastCheckinExpiredAt': 0,
    'marked'

#### Perfect! Now time to test you! 

Using `https://api.foursquare.com/v2/venues/explore` base url, set parameters based on the values [here](https://developer.foursquare.com/docs/api/venues/explore)

After you get a JSON file you want, make a function and for loop to sort through the JSON file and make a pandas dataframe with the following columns.
* Venue ID for the objects
* Name of venue
* Phone number 
* City
* Venue URL for their website

##### Good luck!

In [ ]:
## Your Code Here